In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [2]:
from metrics_monitor import monitor_table
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import matthews_corrcoef

In [4]:
df_exames_einstein = pd.read_csv('EINSTEIN_Exames_2.csv', sep='|')

In [13]:
df_pacientes_einstein = pd.read_csv('EINSTEIN_Pacientes_2.csv', sep ='|')

In [ ]:
df_exames_hsl = pd.read_csv('HSL_Exames_2.csv', sep='|')

In [20]:
df_pacientes_hsl = pd.read_csv('HSL_Pacientes_2.csv', sep='|')

In [ ]:
df_exames_fleury = pd.read_csv('GrupoFleury_Exames_2.csv', sep='|')

In [21]:
df_pacientes_fleury = pd.read_csv('GrupoFleury_Pacientes_2.csv', sep='|')

In [ ]:
df_exames_einstein.DE_EXAME.unique()

In [6]:
df_hemogram_einstein1 = df_exames_einstein.loc[df_exames_einstein.DE_EXAME == 'Hemograma com Plaquetas']

In [7]:
df_hemogram_einstein2 = df_exames_einstein.loc[df_exames_einstein.DE_EXAME == 'Hemograma Contagem Auto']

In [8]:
df_hemogram_einstein1.DE_RESULTADO =  pd.to_numeric(df_hemogram_einstein1.DE_RESULTADO, errors='coerce')
df_hemogram_einstein2.DE_RESULTADO =  pd.to_numeric(df_hemogram_einstein2.DE_RESULTADO, errors='coerce')

/home/neto/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [9]:
df_hemogram_einstein1 = df_hemogram_einstein1.pivot_table('DE_RESULTADO', ['ID_PACIENTE', 'DT_COLETA'], 'DE_ANALITO')
df_hemogram_einstein2 = df_hemogram_einstein2.pivot_table('DE_RESULTADO', ['ID_PACIENTE', 'DT_COLETA'], 'DE_ANALITO')

In [13]:
df_hemogram_einstein = pd.merge(df_hemogram_einstein1.reset_index(), df_hemogram_einstein2.reset_index(), 
                               right_on=['ID_PACIENTE', 'DT_COLETA'], left_on=['ID_PACIENTE', 'DT_COLETA'])

In [14]:
df_pcr_einstein = df_exames_einstein.loc[df_exames_einstein.DE_EXAME =='PCR em tempo real para detecção de Coron']

In [15]:
df_einstein = pd.merge(df_pcr_einstein, df_hemogram_einstein,
                               left_on = ['ID_PACIENTE', 'DT_COLETA'], right_on=['ID_PACIENTE', 'DT_COLETA']    )

In [16]:
df_einstein = pd.merge(df_einstein, df_pacientes_einstein, left_on='ID_PACIENTE', right_on = 'ID_PACIENTE')

In [18]:
df_einstein.to_csv('einstein_base_hem_pcr.csv')

In [ ]:
df_hemogram_hsl = df_exames_hsl.loc[df_exames_hsl.DE_EXAME == 'Hemograma, sangue total']

In [ ]:
df_hemogram_hsl = df_hemogram_hsl.append(df_exames_hsl.loc[df_exames_hsl.DE_EXAME == 'Hemograma'])

In [ ]:
df_hemogram_hsl.DE_RESULTADO = df_hemogram_hsl.DE_RESULTADO.str.replace(',', '.')


df_hemogram_hsl.DE_RESULTADO =  pd.to_numeric(df_hemogram_hsl.DE_RESULTADO, errors='coerce')

In [ ]:
df_hemogram_hsl = df_hemogram_hsl.pivot_table('DE_RESULTADO', ['ID_PACIENTE', 'DT_COLETA'], 'DE_ANALITO').reset_index()

In [ ]:
df_pcr_hsl = df_exames_hsl.loc[df_exames_hsl.DE_EXAME == 'COVID-19-PCR para SARS-COV-2, Vários Materiais (Fleury)']

In [ ]:
df_pcr_hsl = df_pcr_hsl.append(df_exames_hsl.loc[df_exames_hsl.DE_EXAME == 'Detecção de Coronavírus (NCoV-2019) POR PCR (Anatomia Patológica)'])

In [ ]:
df_hsl= pd.merge(df_pcr_hsl, df_hemogram_hsl, left_on = ['ID_PACIENTE', 'DT_COLETA'], right_on=['ID_PACIENTE', 'DT_COLETA']    )

In [ ]:
df_hsl = pd.merge(df_hsl, df_pacientes_hsl, left_on='ID_PACIENTE', right_on='ID_PACIENTE')

In [ ]:
df_exames_fleury.DE_EXAME.unique()

In [ ]:
df_hemogram_fleury = df_exames_fleury.loc[df_exames_fleury.DE_EXAME == 'HEMOGRAMA, sangue total']

In [ ]:
df_hemogram_fleury.DE_RESULTADO = df_hemogram_fleury.DE_RESULTADO.str.replace(',', '.')

df_hemogram_fleury.DE_RESULTADO =  pd.to_numeric(df_hemogram_fleury.DE_RESULTADO, errors='coerce')

In [ ]:
df_pcr_fleury = df_exames_fleury.loc[df_exames_fleury.DE_EXAME ==  'NOVO CORONAVÍRUS 2019 (SARS-CoV-2), DETECÇÃO POR PCR']

In [ ]:
df_fleury = pd.merge(df_pcr_fleury, df_hemogram_fleury.pivot_table('DE_RESULTADO', ['ID_PACIENTE', 'DT_COLETA'], 'DE_ANALITO').reset_index(),
                               left_on = ['ID_PACIENTE', 'DT_COLETA'], right_on=['ID_PACIENTE', 'DT_COLETA'] )

In [ ]:
df_pacientes_fleury.columns

In [ ]:
df_fleury = pd.merge(df_fleury, df_pacientes_fleury, left_on=['ID_PACIENTE'], right_on=['ID_PACIENTE'])

In [19]:
df_einstein.DE_RESULTADO.value_counts()

Não detectado    7565
Detectado        2611
nova coleta         1
Inconclusivo        1
Name: DE_RESULTADO, dtype: int64

In [20]:
df_einstein = df_einstein.loc[df_einstein.DE_RESULTADO.isin(['Não detectado', 'Detectado'])]

In [ ]:
df_fleury.DE_RESULTADO.unique()

In [ ]:
df_fleury = df_fleury.loc[df_fleury.DE_RESULTADO.isin(['NÃO DETECTADO', 'NÃO DETECTADO (NEGATIVO)','DETECTADO (POSITIVO)',
                                                              'DETECTADO'])]

In [ ]:
df_hsl.DE_RESULTADO.unique()


In [ ]:
df_hsl = df_hsl.loc[df_hsl.DE_RESULTADO.isin(['DETECTADO (POSITIVO)', 'DETECTADO', 'NÃO DETECTADO (NEGATIVO)',
                                                     'NÃO DETECTADO'])]

In [ ]:
df_hsl.DE_RESULTADO.unique()

In [ ]:
df_einstein.columns

In [2]:
df_einstein = pd.read_csv('einstein_filtrado_1ocorrencia.csv')

In [4]:
df_fleury = pd.read_csv('fleury_filtrado_1ocorrencia.csv')

In [5]:
df_hsl = pd.read_csv('hsl_filtrado_1ocorrencia.csv')

In [2]:
df_einstein = pd.read_csv('df_hem_pcr_einstein1ocorrencia.csv')

In [3]:
df_fleury = pd.read_csv('df_hem_pcr_fleury1ocorrencia.csv')

In [4]:
df_hsl = pd.read_csv('df_hem_pcr_hsl1ocorrencia.csv')

In [5]:
df_einstein.columns

Index(['ID_PACIENTE', 'DT_COLETA', 'Basófilos', 'Basófilos #', 'Eosinófilos',
       'Eosinófilos  #', 'Linfócitos', 'Linfócitos #', 'Monócitos',
       'Monócitos #', 'Neutrófilos', 'Neutrófilos  #', 'CHCM', 'HCM',
       'Hematócrito', 'Hemoglobina', 'Hemácias', 'Leucócitos', 'Leucócitos #',
       'Plaquetas', 'RDW', 'VCM', 'Volume Médio Plaquetário', 'DE_ORIGEM',
       'DE_EXAME', 'DE_ANALITO', 'DE_RESULTADO', 'CD_UNIDADE',
       'DE_VALOR_REFERENCIA'],
      dtype='object')

In [6]:
df_einstein.DE_RESULTADO.unique()

array(['Não detectado', 'Detectado', 'nova coleta'], dtype=object)

In [7]:
df_einstein = df_einstein.loc[df_einstein.DE_RESULTADO.isin(['Não detectado', 'Detectado'])]

In [8]:
df_einstein['DE_RESULTADO'] = df_einstein.DE_RESULTADO.replace(['Não detectado', 'Detectado'], [0, 1])

<ipython-input-8-68801dd2ada8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_einstein['DE_RESULTADO'] = df_einstein.DE_RESULTADO.replace(['Não detectado', 'Detectado'], [0, 1])


In [9]:
df_hsl.DE_RESULTADO.unique()

array(['DETECTADO', 'NÃO DETECTADO (NEGATIVO)'], dtype=object)

In [10]:
df_hsl.DE_RESULTADO = df_hsl.DE_RESULTADO.replace(['NÃO DETECTADO (NEGATIVO)', 'DETECTADO'], [0, 1])

In [41]:
df_einstein = df_einstein.loc[:,~df_einstein.columns.duplicated()]


In [18]:
df_pacientes_einstein['idade'] = 2020 -  pd.to_numeric(df_pacientes_einstein['AA_NASCIMENTO'], errors='coerce') 

In [22]:
df_pacientes_fleury['idade'] = 2020 -  pd.to_numeric(df_pacientes_fleury['AA_NASCIMENTO'], errors='coerce') 

In [23]:
df_pacientes_hsl['idade'] = 2020 -  pd.to_numeric(df_pacientes_hsl['AA_NASCIMENTO'], errors='coerce') 

In [24]:
df_einstein = pd.merge(df_einstein, df_pacientes_einstein, on=['ID_PACIENTE'])

In [25]:
df_fleury = pd.merge(df_fleury, df_pacientes_fleury, on=['ID_PACIENTE'])

In [26]:
df_hsl = pd.merge(df_hsl, df_pacientes_hsl, on=['ID_PACIENTE'])

In [27]:
df_einstein = df_einstein[['CHCM', 'HCM',
       'Hematócrito', 'Hemoglobina', 'Hemácias', 'Leucócitos', 'Leucócitos #',
       'Plaquetas', 'RDW', 'VCM', 'Volume Médio Plaquetário', 'Basófilos',
       'Basófilos #', 'Eosinófilos', 'Eosinófilos  #', 'Linfócitos',
       'Linfócitos #', 'Monócitos', 'Monócitos #', 'Neutrófilos',
       'Neutrófilos  #', 'DE_RESULTADO', 'idade', 'IC_SEXO', 'DE_RESULTADO']]

In [28]:
df_einstein.rename(columns={'Volume Médio Plaquetário':'Volume plaquetário médio'}, inplace=True)

/home/neto/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [29]:
df_einstein.rename(columns={'Hemácias':'Eritrócitos'}, inplace=True)

In [30]:
df_einstein.rename(columns={'Basófilos': 'Basófilos (%)', 'Leucócitos':'Leucócitos (%)', 
                                   'Eosinófilos': 'Eosinófilos (%)', 'Linfócitos':'Linfócitos (%)',
                                   'Monócitos':'Monócitos (%)', 'Neutrófilos':'Neutrófilos (%)'}, inplace=True)

In [31]:
df_einstein.rename(columns={'Basófilos #': 'Basófilos', 'Leucócitos #':'Leucócitos', 
                                   'Eosinófilos  #':'Eosinófilos','Linfócitos #':'Linfócitos',
                                   'Monócitos #':'Monócitos', 'Neutrófilos  #':'Neutrófilos'}, inplace=True)

In [32]:
#converting platelets unit

df_einstein['Plaquetas'] = df_einstein['Plaquetas'] * (10**3)

<ipython-input-32-05ec0ef1a96d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_einstein['Plaquetas'] = df_einstein['Plaquetas'] * (10**3)


In [61]:
df_einstein = df_einstein.drop_duplicates()

In [56]:
df_fleury.columns.sort_values()

Index(['Basófilos', 'Basófilos (%)', 'CHCM', 'DE_RESULTADO', 'Eosinófilos',
       'Eosinófilos (%)', 'Eritrócitos', 'HCM', 'Hematócrito', 'Hemoglobina',
       'IC_SEXO', 'Leucócitos', 'Linfócitos', 'Linfócitos (%)', 'Monócitos',
       'Monócitos (%)', 'Neutrófilos', 'Neutrófilos (%)', 'Plaquetas', 'RDW',
       'VCM', 'Volume plaquetário médio', 'idade'],
      dtype='object')

In [27]:
df_einstein.to_csv('einstein_base_hem_pcr.csv')

### fleury

In [45]:
set(df_einstein.columns) - set(df_hsl.columns)

{'Leucócitos (%)'}

In [41]:
df_fleury.columns

Index(['Basófilos', 'Basófilos (%)', 'CHCM', 'Eosinófilos', 'Eosinófilos (%)',
       'Eritrócitos', 'Hematócrito', 'Hemoglobina', 'HCM', 'Leucócitos',
       'Linfócitos', 'Linfócitos (%)', 'Monócitos', 'Monócitos (%)',
       'Neutrófilos', 'Neutrófilos (%)', 'Plaquetas', 'RDW', 'VCM',
       'Volume plaquetário médio', 'DE_RESULTADO', 'idade', 'IC_SEXO'],
      dtype='object')

In [33]:
df_fleury = df_fleury[['Basófilos', 'Basófilos (%)', 'Concentração de Hemoglobina Corpuscular', 'Eosinófilos',
       'Eosinófilos (%)', 'Eritrócitos', 'Hematócrito', 'Hemoglobina', 'Hemoglobina Corpuscular Média',
       'Leucócitos', 'Linfócitos', 'Linfócitos (%)','Monócitos',   'Monócitos (%)', 'Neutrófilos', 'Neutrófilos (%)', 'Plaquetas',
       'RDW','VCM','Volume plaquetário médio', 'DE_RESULTADO', 'idade', 'IC_SEXO']]

In [34]:
df_fleury.rename(columns= {'Concentração de Hemoglobina Corpuscular':'CHCM', 
                           'Hemoglobina Corpuscular Média':'HCM'}, inplace=True)

### HSL

In [21]:
df_hsl.columns

Index(['ID_PACIENTE', 'ID_ATENDIMENTO', 'DT_COLETA', 'DE_ORIGEM_x', 'DE_EXAME',
       'DE_ANALITO', 'DE_RESULTADO', 'CD_UNIDADE', 'DE_VALOR_REFERENCIA',
       'Bastonetes', 'Bastonetes (%)', 'Basófilos', 'Basófilos (%)', 'Blastos',
       'Blastos (%)', 'CHCM', 'Eosinófilos', 'Eosinófilos (%)', 'Eritrócitos',
       'Fração Imatura de Plaquetas', 'HCM', 'Hematócrito', 'Hemoglobina',
       'Leucócitos', 'Linfócitos', 'Linfócitos (%)', 'Metamielócitos',
       'Metamielócitos (%)', 'Mielócitos', 'Mielócitos (%)', 'Monócitos',
       'Monócitos (%)', 'Neutrófilos', 'Neutrófilos (%)', 'Plaquetas',
       'Plasmócitos (%)', 'Plasmóticos', 'Promielócitos', 'Promielócitos (%)',
       'RDW', 'Segmentados', 'Segmentados (%)', 'VCM',
       'Volume plaquetário médio', 'Índice de Green & King', 'dimero',
       'DE_ORIGEM_y', 'proteina_c_reativa', 'IC_SEXO', 'idade'],
      dtype='object')

In [35]:
df_hsl = df_hsl[['Basófilos', 'Basófilos (%)', 'CHCM', 'Eosinófilos',
       'Eosinófilos (%)', 'Eritrócitos', 'Hematócrito', 'Hemoglobina', 'HCM',
       'Leucócitos', 'Linfócitos', 'Linfócitos (%)','Monócitos',   'Monócitos (%)', 'Neutrófilos', 'Neutrófilos (%)', 'Plaquetas',
       'RDW','VCM','Volume plaquetário médio', 'DE_RESULTADO', 'idade', 'IC_SEXO'
        ]]

In [ ]:
num_einstein = df_einstein.select_dtypes(include=['float64'])
num_fleury = df_fleury.select_dtypes(include=['float64'])
num_hsl = df_hsl.select_dtypes(include=['float64'])

In [65]:
len(df_einstein.columns)

25

In [68]:
df_einstein.drop(columns='Leucócitos (%)').drop_duplicates().columns

Index(['CHCM', 'HCM', 'Hematócrito', 'Hemoglobina', 'Eritrócitos',
       'Leucócitos', 'Plaquetas', 'RDW', 'VCM', 'Volume plaquetário médio',
       'Basófilos (%)', 'Basófilos', 'Eosinófilos (%)', 'Eosinófilos',
       'Linfócitos (%)', 'Linfócitos', 'Monócitos (%)', 'Monócitos',
       'Neutrófilos (%)', 'Neutrófilos', 'DE_RESULTADO', 'idade', 'IC_SEXO',
       'DE_RESULTADO'],
      dtype='object')

In [70]:
df_einstein = df_einstein.loc[:,~df_einstein.columns.duplicated()]

In [71]:
df_final  = pd.concat([df_einstein.drop(columns='Leucócitos (%)'),df_hsl,df_fleury], axis=0)

In [72]:
df_final

,CHCM,HCM,Hematócrito,Hemoglobina,Eritrócitos,Leucócitos,Plaquetas,RDW,VCM,Volume plaquetário médio,...,Eosinófilos,Linfócitos (%),Linfócitos,Monócitos (%),Monócitos,Neutrófilos (%),Neutrófilos,DE_RESULTADO,idade,IC_SEXO
0,34.7,31.6,40.6,14.1,4.46,4850.0,222000.0,11.6,91.0,8.4,...,184.0,24.4,1183.0,8.1,393.0,63.5,3080.0,0,52.0,M
1,33.9,30.9,40.4,13.7,4.44,5280.0,278000.0,11.9,91.0,10.2,...,79.0,26.7,1410.0,8.5,449.0,62.9,3321.0,0,31.0,F
2,32.6,26.4,39.9,13.0,4.93,4980.0,223000.0,14.4,80.9,10.6,...,0.0,32.9,1638.0,4.4,219.0,62.5,3112.0,1,51.0,F
3,35.1,30.7,47.9,16.8,5.48,7380.0,164000.0,12.9,87.4,11.0,...,59.0,28.9,2133.0,9.3,686.0,60.5,4465.0,0,65.0,M
4,35.2,30.2,41.5,14.6,4.84,4440.0,118000.0,11.8,85.7,11.2,...,178.0,48.3,2145.0,12.2,542.0,35.1,1558.0,1,61.0,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2166,33.7,30.8,40.1,13.5,4.38,6610.0,269000.0,12.7,91.6,10.4,...,120.0,33.0,2180.0,6.5,430.0,58.2,3850.0,0,43.0,F
2167,35.1,32.0,42.2,14.8,4.62,5030.0,206000.0,11.9,91.3,10.9,...,190.0,41.7,2100.0,9.7,490.0,44.0,2210.0,0,39.0,M
2168,34.4,29.2,38.7,13.3,4.56,4850.0,201000.0,12.7,84.9,11.3,...,70.0,29.3,1420.0,15.3,740.0,53.4,2590.0,0,58.0,F
2169,33.3,28.4,40.6,13.5,4.75,5180.0,265000.0,12.5,85.5,9.2,...,160.0,31.1,1610.0,5.4,280.0,59.2,3070.0,0,25.0,F


In [73]:
df_final.to_csv('1ocorrencia_hem_pcr_total.csv', index=False)

In [53]:
set(df_einstein.columns) - set(df_final.columns) 

{'Leucócitos (%)'}

In [50]:
df_final.to_csv('database_filtrado_1ocorrencia.csv', index=False)

In [ ]:
num_einstein.describe()

In [ ]:
num_hsl.describe()

In [4]:
df = pd.read_csv('database_filtrado_1ocorrencia.csv')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2637 entries, 0 to 2636
Data columns (total 26 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Basófilos                 2637 non-null   float64
 1   Basófilos (%)             2637 non-null   float64
 2   CHCM                      2637 non-null   float64
 3   Eosinófilos               2637 non-null   float64
 4   Eosinófilos (%)           2637 non-null   float64
 5   Eritrócitos               2637 non-null   float64
 6   Hematócrito               2637 non-null   float64
 7   Hemoglobina               2637 non-null   float64
 8   HCM                       2637 non-null   float64
 9   Leucócitos                2637 non-null   float64
 10  Linfócitos                2637 non-null   float64
 11  Linfócitos (%)            2637 non-null   float64
 12  Monócitos                 2637 non-null   float64
 13  Monócitos (%)             2637 non-null   float64
 14  Neutrófi

### matriz de correlação dos features do hemograma p cada hospital

In [ ]:

plt.subplots(figsize=(15,10))
corrMatrix = num_einstein.corr()
sns.heatmap(corrMatrix, annot=True)
plt.title('einstein corr matrix')
plt.show()

In [ ]:
plt.subplots(figsize=(15,10))
corrMatrix = num_fleury.corr()
sns.heatmap(corrMatrix, annot=True)
plt.title('fleury corr matrix')
plt.show()

In [ ]:
plt.subplots(figsize=(15,10))
corrMatrix = num_hsl.corr()
sns.heatmap(corrMatrix, annot=True)
plt.title('hsl corr matrix')
plt.show()

In [ ]:
df = pd.concat([df_einstein, df_fleury, df_hsl])

In [147]:
df.to_csv('hemogramas_pcr_einstein_fleury_hsl.csv')

In [ ]:
num_df = df.select_dtypes(include=['float64'])

In [ ]:
plt.subplots(figsize=(16,12))
corrMatrix = num_df.corr()
sns.heatmap(corrMatrix, annot=True)
plt.title('total corr matrix')
plt.show()

## Model

excluding highly correlated features

In [ ]:
df.columns

In [ ]:
df['idade'] = 2020 - pd.to_numeric(df['AA_NASCIMENTO'], errors='coerce')

In [ ]:
df['resultado'] = df.DE_RESULTADO.replace(['Não detectado', 'Detectado', 'NÃO DETECTADO',
       'NÃO DETECTADO (NEGATIVO)', 'DETECTADO', 'DETECTADO (POSITIVO)'],
                                         [0, 1, 0, 0, 1,1])

In [ ]:
df.rename(columns={'IC_SEXO':'sexo'}, inplace=True)

In [ ]:
df = df[['CHCM', 'HCM', 'Hematócrito', 'Hemoglobina', 'Eritrócitos',
        'Leucócitos', 'Plaquetas', 'RDW', 'VCM',
       'Volume plaquetário médio', 'Basófilos',
        'Eosinófilos', 'Linfócitos','Monócitos', 'Neutrófilos',
       'resultado', 'idade', 'sexo']]

In [ ]:
num_df = df.select_dtypes(include=['float64'])

In [ ]:
cat_df = df.select_dtypes(include=['object'])

In [ ]:
numeric_features_list = list(num_df.columns)

In [ ]:
cat_features_list = list(cat_df.columns)

In [ ]:
plt.subplots()
df.resultado.value_counts().plot(kind='bar')

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import RobustScaler
from sklearn.impute import SimpleImputer

In [ ]:
df['sexo'] = df['sexo'].replace(['M', 'F'], [0, 1])

In [ ]:
numeric_transformer = Pipeline(steps = [('imputer', SimpleImputer(strategy='median')),
                                        ('scaler', RobustScaler())
                                            ])

preprocessor = ColumnTransformer(
                        transformers = [
                                ('num', numeric_transformer, numeric_features_list),
                        ])

In [ ]:
X = df.drop(columns='resultado')

y = df['resultado']

X_train, X_test, y_train, y_test = train_test_split( X, y , test_size = 0.33 , random_state = 42, stratify=y )

# RandomForest - Randomized Grid Search

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV,  RandomizedSearchCV


In [ ]:
# Number of trees in Random Forest
rf_n_estimators = [int(x) for x in np.linspace(20, 1000, 10)]


# Maximum number of levels in tree
rf_max_depth = [int(x) for x in np.linspace(5, 55, 11)]
# Add the default as a possible value
rf_max_depth.append(None)

# Number of features to consider at every split
rf_max_features = ['auto', 'sqrt', 'log2']

# Criterion to split on
rf_criterion = ['gini', 'entropy']

# Minimum number of samples required to split a node
rf_min_samples_split = [int(x) for x in np.linspace(2, 10, 9)]

# Minimum decrease in impurity required for split to happen
rf_min_impurity_decrease = [0.0, 0.05, 0.1]

# Method of selecting samples for training each tree
rf_bootstrap = [True, False]

# Create the grid
rf_grid = {'estimator__n_estimators': rf_n_estimators,
               'estimator__max_depth': rf_max_depth,
               'estimator__max_features': rf_max_features,
               'estimator__criterion': rf_criterion,
               'estimator__min_samples_split': rf_min_samples_split,
               'estimator__min_impurity_decrease': rf_min_impurity_decrease,
               'estimator__bootstrap': rf_bootstrap}

In [133]:
# Create the model to be tuned
# rf_base = RandomForestClassifier()

clf_rf = RandomForestClassifier(class_weight='balanced')       

pipe_rf = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('estimator', clf_rf)
])





# Create the random search Random Forest
rf_random = RandomizedSearchCV(estimator = pipe_rf, param_distributions = rf_grid, 
                               n_iter = 100, cv = 10, verbose = 2, random_state = 42, 
                               n_jobs = -1, scoring='recall')

# Fit the random search model
rf_random.fit(X_train, y_train)

# View the best parameters from the random search
rf_random.best_params_

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 11.0min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed: 20.2min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed: 31.7min
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed: 50.7min finished


{'estimator__n_estimators': 20,
 'estimator__min_samples_split': 5,
 'estimator__min_impurity_decrease': 0.1,
 'estimator__max_features': 'log2',
 'estimator__max_depth': 50,
 'estimator__criterion': 'gini',
 'estimator__bootstrap': False}

In [134]:
rf_random.best_params_

{'estimator__n_estimators': 20,
 'estimator__min_samples_split': 5,
 'estimator__min_impurity_decrease': 0.1,
 'estimator__max_features': 'log2',
 'estimator__max_depth': 50,
 'estimator__criterion': 'gini',
 'estimator__bootstrap': False}

In [135]:
pipe_rf_final = rf_random.best_estimator_

In [136]:
pipe_rf_final.fit(X_train, y_train)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  ['CHCM', 'HCM', 'Hematócrito',
                                                   'Hemoglobina', 'Eritrócitos',
                                                   'Leucócitos', 'Plaquetas',
                                                   'RDW', 'VCM',
                                                   'Volume plaquetário médio',
                                                   'Basófilos', 'Eosinófilos',
                                                   'Linfócitos', 'Monócitos',
                            

In [130]:
def performance_test(model_name, model, x_test, y_test):
    # predcitions
    yhat = model.predict(x_test)
    
    # Performance metrics
    precision = precision_score(y_test, yhat)
    recall = recall_score(y_test, yhat)
    f1 = f1_score(y_test, yhat)
    roc_auc = roc_auc_score(y_test, yhat)
    mcc = matthews_corrcoef(y_test, yhat)

    # Performance dataframe
    performance_df = pd.DataFrame({'precision': precision, 'recall': recall, 'f1_score': f1, 
                               'roc_auc_score': roc_auc, 'mcc': mcc}, index=[model_name])

    # confusion matrix
    #cm = confusion_matrix(y_test, yhat)

    # dataframe for heatmap
    #cm_df = pd.DataFrame(cm, columns=['predicted_not_covid', 'predicted_covid'], 
    #                     index=['actual_not_covid', 'actual_covid'])
    
    return performance_df
#, cm_df

In [140]:
performance_test('RandomForest', pipe_rf_final, X_test, y_test)

,precision,recall,f1_score,roc_auc_score,mcc
RandomForest,0.000,0.000,0.000,0.500,0.000


In [ ]:
monitor_table(y_test, pipe_rf_final.predict(X_test), pipe_rf_final.predict_proba(X_test)[:,1])

# XGBoost

In [ ]:
# Number of trees to be used
xgb_n_estimators = [int(x) for x in np.linspace(20, 1000, 10)]

# Maximum number of levels in tree
xgb_max_depth = [int(x) for x in np.linspace(2, 20, 10)]

# Minimum number of instaces needed in each node
xgb_min_child_weight = [int(x) for x in np.linspace(1, 10, 10)]

# Tree construction algorithm used in XGBoost
xgb_tree_method = ['auto', 'exact', 'approx', 'hist', 'gpu_hist']

# Learning rate
xgb_eta = [x for x in np.linspace(0.1, 0.6, 6)]

# Minimum loss reduction required to make further partition
xgb_gamma = [int(x) for x in np.linspace(0, 0.5, 6)]



# Create the grid
xgb_grid = {'estimator__n_estimators': xgb_n_estimators,
            'estimator__max_depth': xgb_max_depth,
            'estimator__min_child_weight': xgb_min_child_weight,
            'estimator__tree_method': xgb_tree_method,
            'estimator__eta': xgb_eta,
            'estimator__gamma': xgb_gamma}

In [ ]:
from xgboost import XGBClassifier

In [ ]:
# Create the model to be tuned
clf_xgb= XGBClassifier(scale_pos_weight=y.value_counts()[0]/y.value_counts()[1], objective='binary:logistic')


pipe_xgb = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('estimator', clf_xgb)
    ])



# Create the random search Random Forest
xgb_random = RandomizedSearchCV(estimator = pipe_xgb, param_distributions = xgb_grid, 
                                n_iter = 200, cv = 10, verbose = 2, 
                                random_state = 420, n_jobs = -1)

# Fit the random search model
xgb_random.fit(X_train, y_train)

# Get the optimal parameters
xgb_random.best_params_

In [ ]:
xgb_random.best_estimator_

In [ ]:

pipe_xgb_final = xgb_random.best_estimator_


In [ ]:
pipe_xgb_final.fit(X_train, y_train)

# Model Performance

In [125]:
performance_test('xgboost', pipe_xgb_final, X_test, y_test)

,precision,recall,f1_score,roc_auc_score,mcc
xgboost,0.650,0.624,0.636,0.746,0.499


In [138]:
from metrics_monitor import monitor_table

In [139]:
monitor_table(y_test, pipe_xgb_final.predict(X_test), pipe_xgb_final.predict_proba(X_test)[:,1])

,value
metric,
Ground Truth Prevalence,0.280
Test set Prevalence,0.269
True positive rate (recall),0.624
False positive rate,0.131
Positive predictive value (precision),0.650
True negative rate (specificity),0.869
False negative rate,0.376
Negative predictive value,0.856
Positive likelihood ratio (bigger is better),4.770
